#### Produce cleaning procedure locally on a small sample

In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
from pandas.tseries import offsets
from pandas.tseries.offsets import *

DEVELOPE = True

### Step1 download and read data
Save a smaller sample when developing

#### cells above should be resumed when download original data

In [2]:
df = pd.read_csv('yellow_tripdata_2015_short.csv', index_col=0)
print ("Shape of data\n{}".format(df.shape))
df.head()

Shape of data
(770806, 18)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
61599709,1,2015-05-12 17:46:38,2015-05-12 18:18:09,1,9.10,-73.863525,40.770012,1,N,-73.982628,40.736942,1,30.0,1.0,0.5,7.45,5.54,44.79
24871662,1,2015-02-27 06:03:22,2015-02-27 06:09:52,1,2.60,-74.008308,40.714573,1,N,-73.988518,40.748821,1,9.0,0.5,0.5,1.50,0.00,11.80
19662635,2,2015-02-15 11:23:13,2015-02-15 11:29:37,1,1.28,-73.956635,40.775738,1,N,-73.967468,40.761639,2,7.0,0.0,0.5,0.00,0.00,7.80
15273088,2,2015-02-05 18:36:07,2015-02-05 18:36:40,1,0.00,-73.974190,40.754951,5,N,-73.974167,40.754822,1,52.0,0.0,0.5,11.47,5.33,69.60
67295899,2,2015-06-06 22:18:20,2015-06-06 22:40:32,3,4.55,-73.973450,40.784496,1,N,-73.993279,40.732471,2,17.5,0.5,0.5,0.00,0.00,18.80


In [3]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'],
      dtype='object')

In [4]:
df = df.drop(['VendorID', 'tpep_dropoff_datetime', 'trip_distance',
         'RateCodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
         'tip_amount', 'tolls_amount', 'total_amount'], axis=1)
df.dropna(inplace=True)
df.head()

,tpep_pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
61599709,2015-05-12 17:46:38,1,-73.863525,40.770012,-73.982628,40.736942
24871662,2015-02-27 06:03:22,1,-74.008308,40.714573,-73.988518,40.748821
19662635,2015-02-15 11:23:13,1,-73.956635,40.775738,-73.967468,40.761639
15273088,2015-02-05 18:36:07,1,-73.974190,40.754951,-73.974167,40.754822
67295899,2015-06-06 22:18:20,3,-73.973450,40.784496,-73.993279,40.732471


In [5]:
df.shape

(770806, 6)

#### borders of NY City
city_long_border = (-74.03, -73.75);
city_lat_border = (40.63, 40.85) 

In [6]:
df = df[df['pickup_longitude'] <= -73.75]
df = df[df['pickup_longitude'] >= -74.03]
df = df[df['pickup_latitude'] <= 40.85]
df = df[df['pickup_latitude'] >= 40.63]
df = df[df['dropoff_longitude'] <= -73.75]
df = df[df['dropoff_longitude'] >= -74.03]
df = df[df['dropoff_latitude'] <= 40.85]
df = df[df['dropoff_latitude'] >= 40.63]
df.shape

(746585, 6)

#### extract all the time features out

In [7]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y/%m/%d %H:%M:%S')

In [8]:
df.dtypes

tpep_pickup_datetime    datetime64[ns]
passenger_count                  int64
pickup_longitude               float64
pickup_latitude                float64
dropoff_longitude              float64
dropoff_latitude               float64
dtype: object

In [9]:
df['year'] = df['tpep_pickup_datetime'].dt.year
df['month'] = df['tpep_pickup_datetime'].dt.month
df['hour'] = df['tpep_pickup_datetime'].dt.hour
df['dayofweek'] = df['tpep_pickup_datetime'].dt.weekday_name
df['year_quarter'] = df['tpep_pickup_datetime'].dt.quarter
df['days_in_month'] = df['tpep_pickup_datetime'].dt.days_in_month

In [15]:
df.head()

,tpep_pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,hour,dayofweek,year_quarter,days_in_month
61599709,2015-05-12 17:46:38,1,-73.863525,40.770012,-73.982628,40.736942,2015,5,17,Tuesday,2,31
24871662,2015-02-27 06:03:22,1,-74.008308,40.714573,-73.988518,40.748821,2015,2,6,Friday,1,28
19662635,2015-02-15 11:23:13,1,-73.956635,40.775738,-73.967468,40.761639,2015,2,11,Sunday,1,28
15273088,2015-02-05 18:36:07,1,-73.974190,40.754951,-73.974167,40.754822,2015,2,18,Thursday,1,28
67295899,2015-06-06 22:18:20,3,-73.973450,40.784496,-73.993279,40.732471,2015,6,22,Saturday,2,30


In [17]:
df.columns

Index(['tpep_pickup_datetime', 'passenger_count', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'year',
       'month', 'hour', 'dayofweek', 'year_quarter', 'days_in_month'],
      dtype='object')

#### For the holiday:
#### I tried to extract whether certain day is the day before a holiday, which I expect lots of people will rush to airport. However I fail to use the 'Holidays / Holiday Calendars' class in pandas which contains 'USFederalHolidayCalendar'.

In [16]:
from pandas.tseries.holiday import USFederalHolidayCalendar
USFederalHolidayCalendar.rules

[Holiday: New Years Day (month=1, day=1, observance=<function nearest_workday at 0x10b6706a8>),
 Holiday: Dr. Martin Luther King Jr. (month=1, day=1, offset=<DateOffset: kwds={'weekday': MO(+3)}>),
 Holiday: Presidents Day (month=2, day=1, offset=<DateOffset: kwds={'weekday': MO(+3)}>),
 Holiday: MemorialDay (month=5, day=31, offset=<DateOffset: kwds={'weekday': MO(-1)}>),
 Holiday: July 4th (month=7, day=4, observance=<function nearest_workday at 0x10b6706a8>),
 Holiday: Labor Day (month=9, day=1, offset=<DateOffset: kwds={'weekday': MO(+1)}>),
 Holiday: Columbus Day (month=10, day=1, offset=<DateOffset: kwds={'weekday': MO(+2)}>),
 Holiday: Veterans Day (month=11, day=11, observance=<function nearest_workday at 0x10b6706a8>),
 Holiday: Thanksgiving (month=11, day=1, offset=<DateOffset: kwds={'weekday': TH(+4)}>),
 Holiday: Christmas (month=12, day=25, observance=<function nearest_workday at 0x10b6706a8>)]